## Current Formulation

In [26]:
%%file sample-formulation.lp

#const intlen = 32.

register(eax;ebx;ecx;edx;esi;edi;edp;esp;eip).
flag(n;z;p).
pos(0..intlen-1).
bit(0;1).
imm8(-128..127).
time(T) :- T = 0..maxtime.
regbit(0, Reg, Pos, 0) :- register(Reg), pos(Pos).
flg(0, Flag, 0) :- flag(Flag).
    
:- regbit(T, Reg, Pos, B1), regbit(T, Reg, Pos, B2), B1 > B2, register(Reg), pos(Pos), bit(B1), bit(B2), time(T).
:- flg(T, Flag, B1), flg(T, Flag, B2), B1 > B2, flag(Flag), bit(B1), bit(B2), time(T).
    
cmp(PC) :- cmp_imm(PC, Reg, Val), register(Reg), imm8(Val).
%cmp(PC) :- cmp_reg(PC, Reg1, Reg2), register(Reg1), register(Reg2).
    
%ge_reg_imm(T, Reg, Val) :- time(T), register(Reg), imm8(Val), gt_reg_imm(T, Reg, Val).
%ge_reg_imm(T, Reg, Val) :- time(T), register(Reg), imm8(Val), eq_reg_imm(T, Reg, Val).
%le_reg_imm(T, Reg, Val) :- time(T), register(Reg), imm8(Val), gt_reg_imm(T, Reg, Val).
%le_reg_imm(T, Reg, Val) :- time(T), register(Reg), imm8(Val), eq_reg_imm(T, Reg, Val).
%ne_reg_imm(T, Reg, Val) :- time(T), register(Reg), imm8(Val), gt_reg_imm(T, Reg, Val).
%ne_reg_imm(T, Reg, Val) :- time(T), register(Reg), imm8(Val), lt_reg_imm(T, Reg, Val).

%gt_reg_imm(T, Reg, Val) :-
%    time(T),
%    register(Reg),
%    imm8(Val),
%    regbit(T, Reg, Pos, 1),
%    pos(Pos),
%    Pos > @bitlen(Val).
%gt_reg_imm(T, Reg, Val) :-
%    time(T),
%    register(Reg),
%    imm8(Val),
%    regbit(T, Reg, Pos, 1),
%    pos(Pos),
%    Pos = @bitlen(Val),
%    regbit(T, Reg, Pos2, 

flg(T, Flag, Val) :-
    time(T),
    flag(Flag),
    regval(T, eip, PC),
    flg(T-1, Flag, Val),
    not cmp(PC).

flg(T, n, 0) :-
    time(T),
    flag(Flag),
    regval(T, eip, PC),
    cmp_imm(PC, Reg, Val),
    register(Reg),
    imm8(Val),
    regval(T, Reg, RVal),
    RVal >= Val.
    
flg(T, n, 1) :-
    time(T),
    flag(Flag),
    regval(T, eip, PC),
    cmp_imm(PC, Reg, Val),
    register(Reg),
    imm8(Val),
    regval(T, Reg, RVal),
    RVal < Val.
    
flg(T, z, 0) :-
    time(T),
    flag(Flag),
    regval(T, eip, PC),
    cmp_imm(PC, Reg, Val),
    register(Reg),
    imm8(Val),
    regval(T, Reg, RVal),
    RVal != Val.
    
flg(T, z, 1) :-
    time(T),
    flag(Flag),
    regval(T, eip, PC),
    cmp_imm(PC, Reg, Val),
    register(Reg),
    imm8(Val),
    regval(T, Reg, RVal),
    RVal = Val.
    
flg(T, p, 0) :-
    time(T),
    flag(Flag),
    regval(T, eip, PC),
    cmp_imm(PC, Reg, Val),
    register(Reg),
    imm8(Val),
    regval(T, Reg, RVal),
    RVal <= Val.
    
flg(T, p, 1) :-
    time(T),
    flag(Flag),
    regval(T, eip, PC),
    cmp_imm(PC, Reg, Val),
    register(Reg),
    imm8(Val),
    regval(T, Reg, RVal),
    RVal > Val.

regbit(T, Reg, Pos, Val) :-
    time(T),
    register(Reg),
    pos(Pos),
    Reg != eip,
    regval(T, eip, PC),
    regbit(T-1, Reg, Pos, Val),
    Reg != eax,
    not end(PC).
    
regbit(T, eax, Pos, Val) :-
    time(T),
    pos(Pos),
    regval(T, eip, PC),
    regbit(T-1, eax, Pos, Val),
    not apicall(PC, _).
    
regval(T, eip, PC) :-
    time(T),
    regval(T-1, eip, PC-1),
    %not jmp(PC, _),
    %not je(PC, _),
    not jne(PC-1, _),
    %not jg(PC, _),
    %not jge(PC, _),
    %not jl(PC, _),
    %not jle(PC, _),
    not end(PC-1).
    
regbit(T, eip, Pos, B) :-
    time(T),
    pos(Pos),
    jne(PC2, PC),
    regval(T-1, eip, PC2),
    PC / (2 ** Pos) & 1 = B,
    flg(T-1, z, 0).
    
regbit(T, eip, Pos, B) :-
    time(T),
    pos(Pos),
    jne(PC-1, _),
    regval(T-1, eip, PC-1),
    PC / (2 ** Pos) & 1 = B,
    flg(T-1, z, 1).

%% Right now, 32 bits takes forever, so currently only allowing 8 bits of entropy.
apicall_res(T, Pos, 0) :-
    Pos > 7,
    time(T),
    pos(Pos),
    regval(T, eip, PC),
    apicall(PC, get_something).
1 { apicall_res(T, Pos, B):bit(B) } 1 :-
    Pos <= 7,
    time(T),
    pos(Pos),
    regval(T, eip, PC),
    apicall(PC, get_something).
    
regbit(T, eax, Pos, B) :-
    time(T),
    pos(Pos),
    apicall_res(T, Pos, B),
    regval(T, eip, PC),
    apicall(PC, _).
    
malware_sig_eicar(T) :-
    time(T),
    regval(T, eip, PC),
    eicar(PC).
    
malware_sig :- malware_sig_eicar(T), time(T).
    
%:- not malware_sig.
    
%#script(python)
%def bitlen(x):
%    return int(x).bit_length()
%#end.

%#show malware_sig_eicar/1.
%#show apicall_res/3.
#show regval/3.
#show apicall_res/3.


Overwriting sample-formulation.lp


### Convert between numeric and bitwise representation

In [23]:
%%file regval32.lp

regbit(T, Reg, Pos, B) :-
    time(T),
    register(Reg),
    pos(Pos),
    regval(T, Reg, Val),
    Val / (2 ** Pos) & 1 = B.

regval(T, Reg, Val) :-
    time(T),
    register(Reg),
    regbit(T, Reg, 0, B0),
    regbit(T, Reg, 1, B1),
    regbit(T, Reg, 2, B2),
    regbit(T, Reg, 3, B3),
    regbit(T, Reg, 4, B4),
    regbit(T, Reg, 5, B5),
    regbit(T, Reg, 6, B6),
    regbit(T, Reg, 7, B7),
    regbit(T, Reg, 8, B8),
    regbit(T, Reg, 9, B9),
    regbit(T, Reg, 10, B10),
    regbit(T, Reg, 11, B11),
    regbit(T, Reg, 12, B12),
    regbit(T, Reg, 13, B13),
    regbit(T, Reg, 14, B14),
    regbit(T, Reg, 15, B15),
    regbit(T, Reg, 16, B16),
    regbit(T, Reg, 17, B17),
    regbit(T, Reg, 18, B18),
    regbit(T, Reg, 19, B19),
    regbit(T, Reg, 20, B20),
    regbit(T, Reg, 21, B21),
    regbit(T, Reg, 22, B22),
    regbit(T, Reg, 23, B23),
    regbit(T, Reg, 24, B24),
    regbit(T, Reg, 25, B25),
    regbit(T, Reg, 26, B26),
    regbit(T, Reg, 27, B27),
    regbit(T, Reg, 28, B28),
    regbit(T, Reg, 29, B29),
    regbit(T, Reg, 30, B30),
    regbit(T, Reg, 31, B31),
    Val = (B31 * (2 ** 31)) +
          (B30 * (2 ** 30)) +
          (B29 * (2 ** 29)) +
          (B28 * (2 ** 28)) +
          (B27 * (2 ** 27)) +
          (B26 * (2 ** 26)) +
          (B25 * (2 ** 25)) +
          (B24 * (2 ** 24)) +
          (B23 * (2 ** 23)) +
          (B22 * (2 ** 22)) +
          (B21 * (2 ** 21)) +
          (B20 * (2 ** 20)) +
          (B19 * (2 ** 19)) +
          (B18 * (2 ** 18)) +
          (B17 * (2 ** 17)) +
          (B16 * (2 ** 16)) +
          (B15 * (2 ** 15)) +
          (B14 * (2 ** 14)) +
          (B13 * (2 ** 13)) +
          (B12 * (2 ** 12)) +
          (B11 * (2 ** 11)) +
          (B10 * (2 ** 10)) +
          (B9 * (2 ** 9)) +
          (B8 * (2 ** 8)) +
          (B7 * (2 ** 7)) +
          (B6 * (2 ** 6)) +
          (B5 * (2 ** 5)) +
          (B4 * (2 ** 4)) +
          (B3 * (2 ** 3)) +
          (B2 * (2 ** 2)) +
          (B1 * (2 ** 1)) + B0.


Overwriting regval32.lp


## Simple Instance with 1 choice

In [29]:
%%file instance-2.lp

#const maxtime = 6.

apicall(1, get_something).
cmp_imm(2, eax, 0).
jne(3, 5).
eicar(4).
end(5).

Overwriting instance-2.lp


In [37]:
!clingo regval32.lp instance-2.lp sample-formulation.lp --time-limit=60 | tail

Reading from regval32.lp ...
Solving...
Answer: 1
apicall_res(1,8,0) apicall_res(1,9,0) apicall_res(1,10,0) apicall_res(1,11,0) apicall_res(1,12,0) apicall_res(1,13,0) apicall_res(1,14,0) apicall_res(1,15,0) apicall_res(1,16,0) apicall_res(1,17,0) apicall_res(1,18,0) apicall_res(1,19,0) apicall_res(1,20,0) apicall_res(1,21,0) apicall_res(1,22,0) apicall_res(1,23,0) apicall_res(1,24,0) apicall_res(1,25,0) apicall_res(1,26,0) apicall_res(1,27,0) apicall_res(1,28,0) apicall_res(1,29,0) apicall_res(1,30,0) apicall_res(1,31,0) regval(0,eip,0) regval(0,esp,0) regval(0,edp,0) regval(0,edi,0) regval(0,esi,0) regval(0,edx,0) regval(0,ecx,0) regval(0,ebx,0) regval(0,eax,0) regval(1,eip,1) regval(2,eip,2) regval(1,esp,0) regval(1,edp,0) regval(1,edi,0) regval(1,esi,0) regval(1,edx,0) regval(1,ecx,0) regval(1,ebx,0) regval(3,eip,3) regval(2,ebx,0) regval(2,ecx,0) regval(2,edx,0) regval(2,esi,0) regval(2,edi,0) regval(2,edp,0) regval(2,esp,0) regval(3,esp,0) regval(3,edp,0) regval(3,edi,0) regval(3

All time is in grounding.

In [36]:
!clingo regval32.lp instance-2.lp sample-formulation.lp --time-limit=60 --stats | tail

Rules        : 26998    (Original: 26990)
  Choice     : 8       
Atoms        : 3285    
Bodies       : 2789     (Original: 2797)
  Count      : 0        (Original: 8)
Equivalences : 5384     (Atom=Atom: 50 Body=Body: 9 Other: 5325)
Tight        : No       (SCCs: 13 Non-Hcfs: 0 Nodes: 4373 Gammas: 0)
Variables    : 1791     (Eliminated:    0 Frozen: 1763)
Constraints  : 48864    (Binary:  96.4% Ternary:   0.6% Other:   3.0%)



In [31]:
%%file instance-4.lp

#const maxtime = 12.

apicall(1, get_something).
cmp_imm(2, eax, 0).
jne(3, 5).
eicar(4).
apicall(5, get_something).
cmp_imm(6, eax, 12).
jne(7, 9).
eicar(8).
end(9).

Overwriting instance-4.lp


In [32]:
!clingo regval32.lp instance-4.lp sample-formulation.lp --time-limit=60 | tail

Reading from regval32.lp ...
Solving...
Answer: 1
apicall_res(1,8,0) apicall_res(1,9,0) apicall_res(1,10,0) apicall_res(1,11,0) apicall_res(1,12,0) apicall_res(1,13,0) apicall_res(1,14,0) apicall_res(1,15,0) apicall_res(1,16,0) apicall_res(1,17,0) apicall_res(1,18,0) apicall_res(1,19,0) apicall_res(1,20,0) apicall_res(1,21,0) apicall_res(1,22,0) apicall_res(1,23,0) apicall_res(1,24,0) apicall_res(1,25,0) apicall_res(1,26,0) apicall_res(1,27,0) apicall_res(1,28,0) apicall_res(1,29,0) apicall_res(1,30,0) apicall_res(1,31,0) regval(0,eip,0) regval(0,esp,0) regval(0,edp,0) regval(0,edi,0) regval(0,esi,0) regval(0,edx,0) regval(0,ecx,0) regval(0,ebx,0) regval(0,eax,0) regval(1,eip,1) regval(2,eip,2) regval(1,esp,0) regval(1,edp,0) regval(1,edi,0) regval(1,esi,0) regval(1,edx,0) regval(1,ecx,0) regval(1,ebx,0) regval(3,eip,3) regval(2,ebx,0) regval(2,ecx,0) regval(2,edx,0) regval(2,esi,0) regval(2,edi,0) regval(2,edp,0) regval(2,esp,0) regval(3,esp,0) regval(3,edp,0) regval(3,edi,0) regval(3

In [33]:
%%file instance-5.lp

#const maxtime = 18.

apicall(1, get_something).
cmp_imm(2, eax, 0).
jne(3, 5).
eicar(4).
apicall(5, get_something).
cmp_imm(6, eax, 12).
jne(7, 9).
eicar(8).
apicall(9, get_something).
cmp_imm(10, eax, 24).
jne(11, 13).
eicar(12).
end(13).

Writing instance-5.lp


In [35]:
!clingo regval32.lp instance-5.lp sample-formulation.lp --time-limit=120 | tail

clingo version 5.1.0
Reading from regval32.lp ...
UNKNOWN

TIME LIMIT   : 1
Models       : 0+
Calls        : 1
Time         : 120.001s (Solving: 0.00s 1st Model: 0.00s Unsat: 0.00s)
CPU Time     : 118.609s


*** Info : (clingo): INTERRUPTED by signal!


This would indicate time is exponential on choice rules. A more efficient formulation is needed.